# Исследование надёжности заёмщиков банка

## План проекта:
### Общая информация о проекте:
 1. [Описание проекта](#1)
 2. [Описание данных](#2)
 ---
### 1. [Изучение данных](#3)
---
### 2. [Предобработка данных](#4)
- [Изучение пропусков в данных](#5)
- [Изменение типов данных](#6)
- [Проверка на дубликаты, обработка дубликатов](#7)
---
### 3. [Исследовательский анализ данных](#8)
- [Изучение целей получения кредитов. Лемматизация](#9)
- [Категоризация данных](#10)

### 5. [Влияние признаков на целевое событие](#11)
- [Влияние семейного положения клиента на возврат кредита в срок](#12)
- [Влияние уровня дохода на возврат кредита в срок](#13)
- [Влияние целей получения займа на возврат кредита в срок ](#14)

### 6. [Общий вывод и рекомендации](#15)

# Описание проекта <a id='1'></a>

Заказчик исследования - кредитный отдел банка. Необходимо исследовать предоставленную банком статистику платежёспособности клиентов и выяснить, как разные факторы влияют на возврат кредита в срок. Результаты исследования будут учтены при построении модели **кредитного скоринга**.


# Описание данных<a id='2'></a>

-	children — количество детей в семье
-	days_employed — общий трудовой стаж в днях
-	dob_years — возраст клиента в годах
-	education — уровень образования клиента
-	education_id — идентификатор уровня образования
-	family_status — семейное положение
-	family_status_id — идентификатор семейного положения
-	gender — пол клиента
-	income_type — тип занятости
-	debt — имел ли задолженность по возврату кредитов
-	total_income — ежемесячный доход
-	purpose — цель получения кредита

# 1. Изучение данных<a id='3'></a>

In [1]:
#импортируем библиотеку pandas
import pandas as pd
#отлючаем уведомления от Pandas
import warnings
warnings.filterwarnings('ignore')
#считаем данные из предоставленного файла
data = pd.read_csv('/datasets/data.csv')
#посмотрим общую информацию
print(data.info()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


In [2]:
# посмотрим срез значений по каждому столбцу
display(data.describe()) 

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [3]:
# посмотрим первые строки таблицы
display(data.head()) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

В предоставленной таблице с данными 12 столбцов и 21525 строк, три типа данных. Названия столбцов однотипные и соответствуют содержанию. Есть пропуски в столбцах 'days_employed' и 'total_income'. В столбцах 'children' и 'days_employed' имеются отрицательные значения, которые, скорее всего, являются технической
ошибкой при заполнении или обработке данных.

# 2. Предобработка данных <a id='4'></a>

## Изучение пропусков в данных <a id='5'></a>

В ходе изучения таблицы пропущенные значения были найдены в столбцах 'days_employed' и 'total_income'. Столбец 'days_employed' не влияет на ход исследования, поэтому просто заполним пропуски медианным значением данного столбца. Столбец 'total_income'потребуется для дальнейшей работы с данными, поэтому сначала сгруппируем данные по столбцу 'income_type', в каждой группе посчитаем медиану и только потом заполним пропуски.

Возможными причинами пропуска в данных могут быть: 
1. техническая ошибка при заполнении или обработке анкеты (данные, например, могли не сохраниться)  
2. намеренное / ненамеренное сокрытие информации клиентами банка (могли просто не помнить на момент заполнения или иметь проблемы с налоговой, черная зарплата)  
3. прочие факторы (не хватило времени для заполнения анкеты, невнимательность и др.)

In [4]:
#проверим уникальные значения столбца 'children'
print(data['children'].unique())
print(data['children'].value_counts())

[ 1  0  3  2 -1  4 20  5]
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


В столбце 'children' встречается 47 отрицательных значений - это похоже на техническую ошибку, поскольку при ручном вводе данных более вероятно ошибиться в самом значении, чем в его знаке. Также возможно анкету заполняли люди, негативно относяящиеся к наличию детей в семье, и "-1" выражает их позицию.
В любом случае эти данные лучше удалить, поскольку трудно предположить, что именно имеется ввиду под "-1" и каким значением это следует заменить.

In [5]:
#удаляем строки с отрицательными значениями
df = data[data['children'] >= 0]
#тестово удаляла отрицательные значения из столбца 'days_employed'
#df = df[df['days_employed'] >= 0]
#сразу сбросим индексы после удаления строк
df.reset_index(inplace = True)

Также в столбце 'children' есть клиенты, у которых 20 детей. Данное значение выглядит неправдоподобно, поскольку вряд ли в такую небольшую выборку клиентов могли попасть сразу 76 клиентов с 20 детьми. Здесь более вероятна техническая ошибка при обработке либо невнимательность при заполнении - к исходному числу вписался ноль. Заменим значение "20" на "2".

In [6]:
#заменяем значения "20" в столбце 'children' на "2"
df.loc[df['children'] ==20,'children'] = 2

In [7]:
#проверим уникальные значения столбца 'children' после удаления строк и изменения значений
print(df['children'].unique())
print(df['children'].value_counts())

[1 0 3 2 4 5]
0    14149
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [8]:
#посмотрим на информацию о таблице после удаления строк
print(df.info()) 
print(df.shape) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21478 entries, 0 to 21477
Data columns (total 13 columns):
index               21478 non-null int64
children            21478 non-null int64
days_employed       19307 non-null float64
dob_years           21478 non-null int64
education           21478 non-null object
education_id        21478 non-null int64
family_status       21478 non-null object
family_status_id    21478 non-null int64
gender              21478 non-null object
income_type         21478 non-null object
debt                21478 non-null int64
total_income        19307 non-null float64
purpose             21478 non-null object
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB
None
(21478, 13)


In [9]:
#посмотрим первые 15 строк таблицы,особое внимание обратим на столбцы 'days_employed' и' total_income', в которых есть пропуски
display(df[['income_type','total_income']].head(15))

,income_type,total_income
0,сотрудник,253875.639453
1,сотрудник,112080.014102
2,сотрудник,145885.952297
3,сотрудник,267628.550329
4,пенсионер,158616.077870
5,компаньон,255763.565419
6,компаньон,240525.971920
7,сотрудник,135823.934197
8,сотрудник,95856.832424
9,сотрудник,144425.938277


In [10]:
#столбец 'days_employed' не будет использоваться для анализа данных в ходе выявления закономерностей
#проверим количество строк с пропущенными значениями в столбце 'days_employed'
print(df['days_employed'].isnull().sum())

2171


In [11]:
#посчитаем медиану столбца 'days_employed' для всех положительных значений и присвоим ее значение пропущенным записям
df_1 = df[df['days_employed']>=0]
median_days_employed = df_1['days_employed'].median() 
#заполним пропуски в 'days_employed' значением медианы
df['days_employed'] = df['days_employed'].fillna(median_days_employed)

In [12]:
#убедимся, что пропущенных значений в столбце 'days_employed' нет
print(df['days_employed'].isnull().sum())

0


In [13]:
#теперь проверим количество строк с пропущенными значениями в столбце 'total_income'
print(df['total_income'].isnull().sum())

2171


In [14]:
#чтобы заполнить пропуски в 'total_income', подсчитаем медиану по данному столбцу для каждой группы в 'income_type' 
#сначала посмотрим какие уникальные значения существуют в столбце 'income_type' 
print(df['income_type' ].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


In [15]:
#соберем уникальные значения 'income_type' в словарь 
income_type_dict = df['income_type']
#удалим дубликаты строк
income_type_dict = income_type_dict.drop_duplicates().reset_index(drop=True)
print(income_type_dict)

0          сотрудник
1          пенсионер
2          компаньон
3        госслужащий
4        безработный
5    предприниматель
6            студент
7          в декрете
Name: income_type, dtype: object


In [16]:
#напишем функцию, которая будет принимать датафрейм и две колонки
#по первой колонке будет происходить группировка, по второй колонке - замена пропусков
#для перебора значений по первой колонке будем использовать созданный словарь
def median_total_income(data, column1, column2):
    #используем ранее созданный словарь income_type_dict, чтобы перебрать значения 'income_type'
        for key in income_type_dict:
            #отфильтруем таблицу
            filt2 = data[data[column1] == key]
            #группируем и считаем медиану
            filt3 = filt2.groupby('income_type').median()[['total_income']]
            #вытаскиваем значение медианы в локальную переменную
            total_median = filt3.iloc[0, 0]
            #заполняем пропуски
            data.loc[(data['income_type'] == key) & (data['total_income'].isna()), 'total_income'] = total_median     
            
#вызываем метод median_total_income(data, column1, column2), чтобы заполнить пропуски   
median_total_income(df, 'income_type', 'total_income')
#проверяем, что пропуск заполнил пропущенные значения - 12 строка
display(df.head(15))    

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [17]:
#убедимся, что пропущенных значений в столбце 'total_income' нет
print(df['total_income'].isnull().sum())

0


### Вывод

В данном разделе были определены пропуски в столбцах 'days_employed' и 'total_income'. Пропуски были заполннены медианными значениями в 'days_employed' и медианными значениями после группировки по 'income_type' в 'total_income'. В качестве метода заполнения пропусков была выбрана медиана, поскольку удаление строк с пропусками могло бы привести к искажению результатов (2171 удаленная строчки составили бы 10.1% от данных - это много).

Также было удалено 47 строк с отрицательным значением из столбца 'children'. В этом же столбце у части клинетов поменяли значения (значение "20" заменили на "2"). 
Столбец 'days_employed' не является важным для данного проекта, однако замечено, что если удалить все строки, где общий стаж имеет отрицательное значение, то строк в таблице останется 3438 (от 21525). Возможно, большая часть данных не валидна вследствие технической ошибки при загрузке/выгрузке. Если бы мне на работе выдали такие данные, я бы обратилась к программисту с просьбой попробовать перепроверить и перевыгрузить.

## Изменение типа данных <a id='6'></a>

In [18]:
#обратимся к информации о типах данных в каждой колонке таблицы
print(df.dtypes) 

index                 int64
children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object


In [19]:
#вещественный тип данных (float64) встречается в столбцах 'days_employed', 'total_income'
#поскольку у нас численные данные, то мыможем использовать для преобразования astype()

df['days_employed'] = pd.to_numeric(df['days_employed']).astype('int')
df['total_income'] = pd.to_numeric(df['total_income']).astype('int')

#проверим, что в столбцах 'days_employed', 'total_income' содержится целочисленный тип данных
print(df.info()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21478 entries, 0 to 21477
Data columns (total 13 columns):
index               21478 non-null int64
children            21478 non-null int64
days_employed       21478 non-null int64
dob_years           21478 non-null int64
education           21478 non-null object
education_id        21478 non-null int64
family_status       21478 non-null object
family_status_id    21478 non-null int64
gender              21478 non-null object
income_type         21478 non-null object
debt                21478 non-null int64
total_income        21478 non-null int64
purpose             21478 non-null object
dtypes: int64(8), object(5)
memory usage: 2.1+ MB
None


### Вывод

В таблице с данными в столбцах 'days_employed' и 'total_income' данные имеют вещественный тип. Поскольку вещественный тип данных (float64) уже вляется числовым и не требует использования метода to_numeric(), был выбран метод astype().

## Проверка на дубликаты, обработка дубликатов <a id='7'></a>

При обработке дубликатов уделим внимание столбцам. необходимым для исследования: 'children', 'family_status',  'purpose', а также столбцам с типом данных 'object': 'education', 'family_status', 'gender', 'income_type' , 'purpose'.
Столбец 'children' содержит числа из определенного диапазона, дополнительной обработки данных не требуется.

Сначала будут выявлены и удалены дубликаты внутри указанных колонок - все значения будет переведены в нижний регистр.
Затем будет проведен поиск полностью одинаковых строк -т  в таблице оставим только уникальные строки, 
все встречающиеся дубликаты будут удалены.

In [20]:
#посмотрим размер таблицы до удалению строк дубликатов
print(df.shape)

(21478, 13)


In [21]:
#проверим уникальные значения столбца 'gender' и частоту встречаемости
print(df['gender'].unique())
print(df['gender'].value_counts())

['F' 'M' 'XNA']
F      14201
M       7276
XNA        1
Name: gender, dtype: int64


In [22]:
#присвоим 'XNA' значение 'M'.
for row in data:
        df.gender[df.gender=='XNA'] = 'M'
        
#убедимся, что присвоение произошло
print(df['gender'].value_counts())

F    14201
M     7277
Name: gender, dtype: int64


In [23]:
#проверим уникальные значения для столбца 'education'
print(df['education'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


In [24]:
#заменим все значения на строковые
df['education'] = df['education'].str.lower()
#убедимся, что повторяющихся значений больше нет
print(df['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


In [25]:
#проверим уникальные значения для столбца 'family_status'
print(df['family_status'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


In [26]:
#заменим все значения на строковые
df['family_status'] = df['family_status'].str.lower()
#убедимся, что все значения теперь в едином формате
print(df['family_status'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


In [27]:
#проверим уникальные значения для столбца 'income_type'
print(df['income_type'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


In [28]:
#проверим уникальные значения для столбца 'purpose'
print(df['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [29]:
#посчитаем количество полных дубликатов строк
print(df.duplicated().sum())

0


In [30]:
#поcмотрим на дублирущиеся строки
print(df.loc[df.duplicated(keep='first'),:])

Empty DataFrame
Columns: [index, children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


In [31]:
#удалим полные дубликаты строк,за исключением первого вхождения
df = df.drop_duplicates(keep='first').reset_index(drop=True)
#проверим размер таблицы после удаления дубликатов строк
print(df.shape)

(21478, 13)


In [32]:
#проверим, что полных дубликатов строк в таблице нет
print(df.duplicated().sum())

0


### Вывод

Для удаления дубликатов данных:
1) были заполнены все значения NaN
2) привела значения в столбцах к нижнему регистру, чтобы не было дублирующихся значений в рамках каждого столбца
2) методом drop_duplicates() удалила все дублирующиеся строки, за исключением первого вхождения

Возможные причины появления дубликатов в данных:
1) отсутствие единого формата данных при заполнении анкеты и вводе их в базу
2) ошибки ввода данных
3) человеческий фактор (одна и та же анкета была внесена в базу несколько раз)


# 1. Исследовательский анализ данных <a id='8'></a>

## Изучение целей получения кредитов. Лемматизация <a id='9'></a>

Проведем лемматизацию столбца 'purpose', чтобы выделить основные цели получения кредита клиентами.

In [33]:
#импортируем pymystem3 и Counter
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

In [34]:
#выведем список уникальных значений столбца
print(df['purpose'].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


In [35]:
#создадим список всех целей для получения кредита
#соберем уникальные значения ''purpose' в словарь 
purpose_dict = df['purpose']
#удалим дубликаты строк
purpose_dict = purpose_dict.drop_duplicates().reset_index(drop=True)
print(purpose_dict)

0                              покупка жилья
1                    приобретение автомобиля
2                 дополнительное образование
3                            сыграть свадьбу
4                          операции с жильем
5                                образование
6                      на проведение свадьбы
7                    покупка жилья для семьи
8                       покупка недвижимости
9          покупка коммерческой недвижимости
10                покупка жилой недвижимости
11    строительство собственной недвижимости
12                              недвижимость
13                строительство недвижимости
14        на покупку подержанного автомобиля
15              на покупку своего автомобиля
16     операции с коммерческой недвижимостью
17          строительство жилой недвижимости
18                                     жилье
19           операции со своей недвижимостью
20                                автомобили
21                     заняться образованием
22        

In [36]:
#пропишем метод, который лемматизирует значения столбца 'purpose' и добавляет их в столбец 'lemmas'
def lemmatize_row(row):
    #для каждого значения из purpose_dict проведем лемматизацию
    for key in purpose_dict:
        if row ==key:
            lemmas = m.lemmatize(key)
            #склеим полученные слова и вернем для внесения в таблицу с данными
            return "".join(lemmas).strip()  

In [37]:
#вызовем метод lemmatize_row(row)
df['lemmas'] = df['purpose'].apply(lemmatize_row)
#проверим как теперь выглядят цели в новом столбце
print(df['lemmas'].head(20))

0                              покупка жилье
1                    приобретение автомобиль
2                              покупка жилье
3                 дополнительный образование
4                            сыграть свадьба
5                              покупка жилье
6                           операция с жилье
7                                образование
8                      на проведение свадьба
9                    покупка жилье для семья
10                      покупка недвижимость
11         покупка коммерческий недвижимость
12                           сыграть свадьба
13                   приобретение автомобиль
14                покупка жилой недвижимость
15    строительство собственный недвижимость
16                              недвижимость
17                строительство недвижимость
18           на покупка подержать автомобиль
19                на покупка свой автомобиль
Name: lemmas, dtype: object


In [38]:
#проверим, что все уникальные значения были лемматизированы
print(df['lemmas'].unique())

['покупка жилье' 'приобретение автомобиль' 'дополнительный образование'
 'сыграть свадьба' 'операция с жилье' 'образование'
 'на проведение свадьба' 'покупка жилье для семья' 'покупка недвижимость'
 'покупка коммерческий недвижимость' 'покупка жилой недвижимость'
 'строительство собственный недвижимость' 'недвижимость'
 'строительство недвижимость' 'на покупка подержать автомобиль'
 'на покупка свой автомобиль' 'операция с коммерческий недвижимость'
 'строительство жилой недвижимость' 'жилье'
 'операция со свой недвижимость' 'автомобиль' 'заниматься образование'
 'сделка с подержанный автомобиль' 'получение образование' 'свадьба'
 'получение дополнительный образование' 'покупка свой жилье'
 'операция с недвижимость' 'получение высокий образование'
 'свой автомобиль' 'сделка с автомобиль' 'профильный образование'
 'высокий образование' 'покупка жилье для сдача' 'на покупка автомобиль'
 'ремонт жилье' 'заниматься высокий образование']


In [39]:
#напишем метод, который оставит в столбце 'purpose' только одно слово
def main_purpose(row):
    for word in row: 
        if 'жилье' in row or 'недвижимость' in row:
            return 'недвижимость'
        if 'образование' in row:
            return 'образование'
        if 'автомобиль' in row or 'машина' in row:
            return 'автомобиль'
        if 'свадьба' in row:
            return 'свадьба'
        return 'прочее'

In [40]:
#применим main_purpose(row), создадим новый столбец, содержащий одно слово в качестве цели получения займа
df['main_purpose'] = df['lemmas'].apply(main_purpose)
#проверим распределение целей по датафрейму
print(df['main_purpose'].value_counts())

недвижимость    10816
автомобиль       4304
образование      4012
свадьба          2346
Name: main_purpose, dtype: int64


In [41]:
#проверим,что метод заполнил все  строки и мы ничего не пропустили
print(df['main_purpose'].value_counts().sum())

21478


### Вывод

Для проведения лемматизации столбца 'purpose':
1) были выбраны уникальные значения данного столбца, из которых был создан отдельный список
2) при помощи функции lemmatize_row(row) была проведена лемматизация всех строк столбца 'purpose'
3) при помощи метода main_purpose(row) в таблицу был добавлен столбец 'main_purpose', содержащий одно слово-описание цели получения кредита. Этотстолбец потребуется для итоговых выводов.
Наибольшим спросом пользуются кредиты с целью приобретения недвижимости.

## Категоризация данных <a id='10'></a>

Для того, чтобы увидеть, каких значений в каждом необходимом для анализа столбце больше, а каких совсем мало, создадим словари по интересующим нас столбцам данных.

In [42]:
#напишем функцию, которая собирает словари
def pandas_dict(col):
    return(col.value_counts().to_dict())

In [43]:
#соберем словарь по столбцу 'family_status'
family_status_dict = pandas_dict(df['family_status'])
display(family_status_dict)

{'женат / замужем': 12351,
 'гражданский брак': 4172,
 'не женат / не замужем': 2808,
 'в разводе': 1191,
 'вдовец / вдова': 956}

In [44]:
#соберем словарь по столбцу 'children'
children_dict = pandas_dict(df['children'])
display(children_dict)

{0: 14149, 1: 4818, 2: 2131, 3: 330, 4: 41, 5: 9}

In [45]:
#соберем словарь по столбцу 'main_purpose'
main_purpose_dict = pandas_dict(df['main_purpose'])
display(main_purpose_dict)

{'недвижимость': 10816,
 'автомобиль': 4304,
 'образование': 4012,
 'свадьба': 2346}

### Вывод

В данном разделе были созданы словари для тех данных, которые необходимы для итоговых выводов.

Для первой таблицы с данными семейному положению и возвратом кредита в срок были использованы готовые категории из исходной таблицы.
Большая часть клиентов имеет одинаковый семейный статус (женат/замужем). В меньшинстве оказываются клиенты, которые находятся в разводе либо потеряли супруга. Для сравнения таких разных по количеству групп категорий попробуем в следующем разделе объединить группы.

Для выделения категорий по количеству детей у клиента, были использованы готовые категории из нужного столбца. Из словаря видно, что клиентов с 4 и 5 детьми слишком мало - меньше 50, а значит эти группы будет неправильно сравнивать с остальными группами, поэтому при построении итоговых таблиц будет произведено их объединение в одну группу.

Для выделения категорий клиентов по цели получения кредита были использованы готовые категории, полученные в ходу лемматизации данных.  Отсутствуют слишком маленькие группы, дополнительного объединения не требуется.

Для выделения категорий клиентов по доходу в следующем разделе будут использованы процентили - это позволит разделить всех на равные группы по доходу.

# 4. Влияние признаков на целевое событие <a id='11'></a>

In [46]:
#соберем финальную таблицу из нужных столбцов
df_final = df[['children', 'gender', 'family_status', 'total_income', 'main_purpose','debt']]
display(df_final.head())

,children,gender,family_status,total_income,main_purpose,debt
0,1,F,женат / замужем,253875,недвижимость,0
1,1,F,женат / замужем,112080,автомобиль,0
2,0,M,женат / замужем,145885,недвижимость,0
3,3,M,женат / замужем,267628,образование,0
4,0,F,гражданский брак,158616,свадьба,0


- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Cоберем сводную таблицy с данными о задолженностях и количестве детей.
Предварительно объединим данные о клиентах с 4 и 5 детьми
Клиентов с 5 детьми мало (9 случаев) относительно выборки и у клиентов с 5 детьми нет задолженностей (в таблицу попадает значение NaN, которое при замене на 0 не позволяет произвести верные вычисления).

In [47]:
def child_group(row):
    if row['children'] == 0:
        return 'нет детей'
    elif row['children'] == 1:
        return '1 ребенок'
    elif row['children'] == 2:
        return '2 ребенка'
    elif row['children'] == 3:
        return '3 ребенка'
    else:
        return '4-5 детей'
    
df_final['child_group'] = df.apply(child_group, axis = 1)
children_pivot = df_final.pivot_table(index='child_group', columns='debt', values='gender', aggfunc='count')
children_pivot.columns = ['no_debt', 'debt']
children_pivot['no_debt%'] = round((children_pivot['no_debt'] / (children_pivot['debt'] + children_pivot['no_debt']))*100,1)
children_pivot['debt_%'] = round((children_pivot['debt'] / (children_pivot['debt'] + children_pivot['no_debt']))*100,1)
display(children_pivot.head(10).sort_values('debt_%'))

,no_debt,debt,no_debt%,debt_%
child_group,,,,
нет детей,13086,1063,92.5,7.5
4-5 детей,46,4,92.0,8.0
3 ребенка,303,27,91.8,8.2
1 ребенок,4374,444,90.8,9.2
2 ребенка,1929,202,90.5,9.5


1. Наиболее высокая вероятность возвращения кредита, если у клиента детей нет (92,5% возвращают кредит в срок). Это вполне объяснимо, поскольку с детьми часто связаны незапланированные траты денег, которые могут влиять на своевременность выплат по кредиту.
2. Среди тех, кто имеет детей наибольшая вероятность задолженности по кредиту у клиентов с двумя детьми.
3. Информацию о клиентах с 4-5 детьми учитывать, скорее всего, не стоит, поскольку группа маленькая и имеющиеся данные вряд ли валидны на фоне остальной выборки.

## Влияние семейного положения клиента на возврат кредита в срок <a id='12'></a>

In [48]:
#соберем первую сводную таблиу с данными о задолженностях и семейном статусе
fs_pivot = df_final.pivot_table(index='family_status', columns='debt', values='gender', aggfunc='count')
fs_pivot.columns = ['no_debt', 'debt']
fs_pivot['no_debt%'] = round((fs_pivot['no_debt'] / (fs_pivot['debt'] + fs_pivot['no_debt']))*100,1)
fs_pivot['debt_%'] = round((fs_pivot['debt'] / (fs_pivot['debt'] + fs_pivot['no_debt']))*100,1)
display(fs_pivot.head(10).sort_values('debt_%'))

,no_debt,debt,no_debt%,debt_%
family_status,,,,
вдовец / вдова,893,63,93.4,6.6
в разводе,1106,85,92.9,7.1
женат / замужем,11421,930,92.5,7.5
гражданский брак,3784,388,90.7,9.3
не женат / не замужем,2534,274,90.2,9.8


In [49]:
#добавим в таблицу df_final колонку с категориями по семейному статусу
def family_status2(row):
    if row['family_status'] in ['не женат / не замужем'] or row['family_status'] in ['гражданский брак']:
        return 'клиент не заключал официальный брак'
    else:
        return 'клиент заключал официальный брак'

df_final['family_status2'] = df.apply(family_status2, axis = 1)
#print(df_final.head())
#соберем вторую сводную таблиу с данными о задолженностях и семейном статусе
fs_pivot2 = df_final.pivot_table(index='family_status2', columns='debt', values='gender', aggfunc='count')
fs_pivot2.columns = ['no_debt', 'debt']
fs_pivot2['no_debt%'] = round((fs_pivot2['no_debt'] / (fs_pivot2['debt'] + fs_pivot2['no_debt']))*100,1)
fs_pivot2['debt_%'] = round((fs_pivot2['debt'] / (fs_pivot2['debt'] + fs_pivot2['no_debt']))*100,1)
display(fs_pivot2.head(10).sort_values('debt_%'))

,no_debt,debt,no_debt%,debt_%
family_status2,,,,
клиент заключал официальный брак,13420,1078,92.6,7.4
клиент не заключал официальный брак,6318,662,90.5,9.5


Если клиент был в официальном браке (замужем/женат, вдоца/вдовец, в разводе), то шанс возвращения кредита в срок выше (92,1%).
При этом клиенты, брак которых был завершен (развод - 92.9%, потеря супруга - 93.4%) имеют наибольшую вероятность вернуть кредит в срок. 
У клиентов, не заключавших официальный брак шанс возвращения кредита - 90,1%.

Такие результаты могут связаны с психологическим фактором - готовностью брать на себя ответственность. Вероятно, клиенты, вступившие в официальный брак, с большей вероятностью готовы брать на себя ответственность за принятые решения.

## Влияние уровня дохода на возврат кредита в срок <a id='13'></a>

In [50]:
#узнаем, какие чему равны процентили в колонке 'total_income'
print(round(data['total_income'].describe()),2)

count      19351.0
mean      167422.0
std       102972.0
min        20667.0
25%       103053.0
50%       145018.0
75%       203435.0
max      2265604.0
Name: total_income, dtype: float64 2


In [51]:
#добавим в таблицу df_final колонку с категориями дохода, разделив на группы по процентилям (25, 50, 75 )
def income_group2(row):
    if  row['total_income'] <= 103053:
        return '0-103053'
    elif  103053 < row['total_income'] <= 145018:
        return '103054 - 145018'
    elif  145018 < row['total_income'] < 203435:
        return '145019 - 203435'
    else:
        return '203435+'
    
df_final['income_groups2'] = df.apply(income_group2, axis = 1)
#print(df_final.head())

#соберем первую сводную таблиу с данными о задолженностях и размере дохода по категориям
ti_pivot2 = df_final.pivot_table(index='income_groups2', columns='debt', values='gender', aggfunc='count')
ti_pivot2.columns = ['no_debt', 'debt']
ti_pivot2['no_debt%'] =round((ti_pivot2['no_debt'] / (ti_pivot2['debt'] + ti_pivot2['no_debt']))*100,1)
ti_pivot2['debt_%'] = round((ti_pivot2['debt'] / (ti_pivot2['debt'] + ti_pivot2['no_debt']))*100,1)
display(ti_pivot2.head(10).sort_values('debt_%'))

,no_debt,debt,no_debt%,debt_%
income_groups2,,,,
203435+,4490,341,92.9,7.1
0-103053,4446,383,92.1,7.9
145019 - 203435,5015,462,91.6,8.4
103054 - 145018,5787,554,91.3,8.7


In [52]:
#разделим всех клиентов на две равные группы по уровню дохода (используем процентиль 50%)
def income_group3(row):
    if  row['total_income'] <= 145018:
        return '0-145018'
    else:
        return '145018+'
    
df_final['income_groups3'] = df.apply(income_group3, axis = 1)

#соберем первую вторую таблиу с данными о задолженностях и размере дохода по категориям
ti_pivot2 = df_final.pivot_table(index='income_groups3', columns='debt', values='gender', aggfunc='count')
ti_pivot2.columns = ['no_debt', 'debt']
ti_pivot2['no_debt%'] =round((ti_pivot2['no_debt'] / (ti_pivot2['debt'] + ti_pivot2['no_debt']))*100,1)
ti_pivot2['debt_%'] = round((ti_pivot2['debt'] / (ti_pivot2['debt'] + ti_pivot2['no_debt']))*100,1)
display(ti_pivot2.head(10).sort_values('debt_%'))

,no_debt,debt,no_debt%,debt_%
income_groups3,,,,
145018+,9505,803,92.2,7.8
0-145018,10233,937,91.6,8.4


Клиенты с высоким (выше 75 процентиль - 92.9%) и низким доходом (ниже 25 процентиль - 92.1%) имеют большую вероятность возвращения кредита в срок. 

Клиенты с высоким уровнем дохода, скорее всего, имеют больший финансовый потенциал для погашения задолженностей в срок.  Для клиентов с низким доходов высокая степень вероятности возвращения кредита в срок может быть обусловлена низким порогом максимального размера кредита, что позволяет клиентам лучше планировать погашение кредита.

Клиенты со средним доходом (диапазон между 25 и 75 процентилями) возвращают кредиты в срок хуже.
В целом, показалось, что зависимость между уровнем дохода и возвращением кредита в срок слабая.

## Влияние целей получения займа на возврат кредита в срок <a id='14'></a>

In [53]:
#соберем сводную таблиу с данными о задолженностях и целях оформления кредита
p_pivot = df_final.pivot_table(index='main_purpose', columns='debt', values='gender', aggfunc='count')
p_pivot.columns = ['no_debt', 'debt']
p_pivot['no_debt%'] =round((p_pivot['no_debt'] / (p_pivot['debt'] + p_pivot['no_debt']))*100,1)
p_pivot['debt_%'] = round((p_pivot['debt'] / (p_pivot['debt'] + p_pivot['no_debt']))*100,1)
display(p_pivot.head(10).sort_values('debt_%'))

,no_debt,debt,no_debt%,debt_%
main_purpose,,,,
недвижимость,10034,782,92.8,7.2
свадьба,2160,186,92.1,7.9
образование,3642,370,90.8,9.2
автомобиль,3902,402,90.7,9.3


Наиболее высокая вероятность возвращения кредита в срок у клиентов, которые оформляют его на недвижимость (92.8%) и проведение свадьбы (92.1%).

Хуже остальных возвращают в срок кредиты, которые связаны с операциями с автомобилем - 90.7%. Возможно это связано с тем, что автомобиль часто требует дополнительных внеплановых расходов на обслуживание.

# Общий вывод и рекомендации <a id='14'></a>

В ходе обработки данных была проделана работа по поиску и заполнению пропусков, удалению неверно заполненных строк, поиску и удалению дубликатов, лемматизации целей получения кредита, выделению категорий данных по каждому запросу.
Смутило, что столбец с общим стажем, хоть он и не оказался нужным, почти весь заполнен отрицательными значениями. Где-то при загрузке/выгрузке данных случилась проблема, которую лучше было бы отловить и отдать в починку.

По результатам можно сделать вывод, что  если у клиента нет детей, либо он состоит в официальном браке, либо планирует потратить деньги на операции с недвижимостью, то шанс возвращения кредита высокий.
Если же у клиента двое детей (либо больше), либо он не состоит в официальном браке, либо берет кредит на операции с автомобилем, то шанс задолженностям по кредиту возрастает.
Самой "плавающей" характеристикой является доход клиента. С одной стороны оказывается, что клиенты с низким доходом (до 25 процентили) возвращают кредиты лучше, чем следующие две группы (25-50/50/75 процентили). Однако, если поднять верхний критерий дохода и включить в группу клиентов с более высоким доходом, то ситуация меняется и шанс возвращения кредита уменьшается. Возможно, уровень дохода  нужно смотреть в связке с каким-то другим критерием.

Зависимости по всем показателям в отдельности не очень сильные. Однако, предположу, что если анализировать каждую анкету как совокупность показателей, то можно будет с большей точностью определять каждого клиента кредитного отдела банка как надежного или ненадежного.